In [77]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from __future__ import print_function
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np

In [78]:
class CharacterTable(object):
    def __init__(self, vocab, maxlen):
        self.vocab = vocab
        self.maxlen = maxlen
    
    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.vocab)))
        for i, c in enumerate(C):
            X[i, c] = 1
        return X
    
    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ','.join(x for x in X)

In [79]:
def generateRandSeq(min, max, len):
    return [np.random.randint(min, max) for _ in range(len)]

In [80]:
TRAINING_SIZE = 150000
TEST_SIZE = 10000
DIGITS = 25
MAXLEN = DIGITS
voc = list(xrange(1000))
ctable = CharacterTable(voc, MAXLEN)

In [82]:
inputs = []
outputs = []
inputs_t = []
outputs_t = []
print('Generating data...')
while len(inputs) < TRAINING_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs.append(s)
    outputs.append([i%200 for i in s])

while len(inputs_t) < TEST_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs_t.append(s)
    outputs_t.append([i%200 for i in s])
print(inputs[12])
print(outputs[12])
print(inputs_t[15])
print(outputs_t[15])

Generating data...
[506, 709, 915, 329, 844, 952, 812, 363, 653, 547, 151, 341, 730, 772, 778, 668, 684, 124, 743, 751, 125, 23, 664, 483, 935]
[106, 109, 115, 129, 44, 152, 12, 163, 53, 147, 151, 141, 130, 172, 178, 68, 84, 124, 143, 151, 125, 23, 64, 83, 135]
[477, 456, 648, 456, 52, 204, 311, 100, 552, 604, 273, 796, 407, 728, 391, 645, 812, 784, 804, 512, 101, 196, 684, 794, 300]
[77, 56, 48, 56, 52, 4, 111, 100, 152, 4, 73, 196, 7, 128, 191, 45, 12, 184, 4, 112, 101, 196, 84, 194, 100]


In [83]:
print('Vectorization...')
X = np.zeros((len(inputs), MAXLEN), dtype=np.int32)
# y = np.zeros((len(outputs), MAXLEN), dtype=np.int32)
y = np.zeros((len(outputs), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs):
    X[i] = inputs[i]

# for i, sentence in enumerate(outputs):
#     y[i] = outputs[i]
for i, sentence in enumerate(outputs):
    y[i] = ctable.encode(sentence, maxlen=MAXLEN)

X_test = np.zeros((len(inputs_t), MAXLEN), dtype=np.int32)
# y_test = np.zeros((len(outputs_t), MAXLEN), dtype=np.int32)
y_test = np.zeros((len(outputs_t), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs_t):
    X_test[i] = inputs_t[i]

# for i, sentence in enumerate(outputs_t):
#     y_test[i] = outputs_t[i]
for i, sentence in enumerate(outputs_t):
    y_test[i] = ctable.encode(sentence, maxlen=MAXLEN)
    
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

Vectorization...
(150000, 25)
(150000, 25, 1000)
(10000, 25)
(10000, 25, 1000)


In [20]:
X[0]

array([28, 27, 81, 75, 78, 11, 27, 43, 13,  2, 96, 43, 20, 62, 63,  4, 33,
       73, 35, 70, 62, 37, 14, 96, 69], dtype=int32)

In [84]:
HIDDEN_SIZE = 256
BATCH_SIZE = 200
LAYERS = 2
'''
Hey guys, I also met this problem and I found this thread. Basically, 
the error info can happen when the dimension of the input data (X_train or Y_train) doesn't match with the 
model's input shape.

In my case (and @LeavesBreathe 's case I guess), the problem is that 
the model is expecting the Y_train to be a 3d tensor. Because of the embedding layer, 
the 2d tensor X_train of size (n_batch, sequence_length) will be eventually converted to a 3d tensor of size 
(n_batch, sequence_length, embedding_size) and will be processed by the succeeding LSTM layer. However, 
the 2d tensor Y_train of size (n_sample, sequence_length) is not converted to 3d, 
which is needed by the decoder LSTM.

To fix this problem, what I did is to convert Y_train into a 3d binary tensor (binary one-hot coding) and it worked.
'''
print('Build model...')
model = Sequential()
model.add(Embedding(len(voc), 300, input_length = MAXLEN))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
for _ in range(LAYERS - 2):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(LSTM(HIDDEN_SIZE))
model.add(RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributedDense(input_dim=HIDDEN_SIZE, output_dim=300))
model.add(Activation('tanh'))
model.add(TimeDistributedDense(input_dim=300, output_dim=len(voc)))
model.add(Activation('softmax'))

model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [85]:
early_stopping = EarlyStopping(monitor='val_loss', patience=6)
hist = model.fit(X, y, batch_size=BATCH_SIZE, nb_epoch=150, 
                 callbacks=[early_stopping],
          validation_split = 0.1, shuffle=True)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 135000 samples, validate on 15000 samples
Epoch 1/150
135000/135000 [==============================] - 374s - loss: 5.3083 - acc: 0.0056 - val_loss: 5.2434 - val_acc: 0.0073
Epoch 2/150
135000/135000 [==============================] - 374s - loss: 5.1445 - acc: 0.0081 - val_loss: 5.0673 - val_acc: 0.0089
Epoch 3/150
135000/135000 [==============================] - 376s - loss: 4.9740 - acc: 0.0116 - val_loss: 4.8680 - val_acc: 0.0137
Epoch 4/150
135000/135000 [==============================] - 374s - loss: 4.7922 - acc: 0.0175 - val_loss: 4.6744 - val_acc: 0.0215
Epoch 5/150
135000/135000 [==============================] - 374s - loss: 4.6167 - acc: 0.0254 - val_loss: 4.4681 - val_acc: 0.0308
Epoch 6/150
135000/135000 [==============================] - 374s - loss: 4.4597 - acc: 0.0333 - val_loss: 4.3366 - val_acc: 0.0310
Epoch 7/150
135000/135000 [==============================] - 374s - loss: 4.3305 - acc: 0.0403 - val_loss: 4.4098 - val_acc: 0.0437
Epoch 8/150
135000/135000

In [64]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

10000/10000 [==============================] - 8s     
Test score: 0.00171989639057
Test accuracy: 0.999540005922
